# Download EMNIST_Byclass dataset with capital letter only

In [1]:
import torch
import random
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import shutil

# Define the augmentation pipeline
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.transpose(Image.FLIP_LEFT_RIGHT)),
    transforms.Lambda(lambda x: x.rotate(90)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# Custom dataset class
class LettersOnlyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Paths
output_folder = '/content/emnist_capitals_dataset'

# EMNIST capital letters (labels 10-35 correspond to A-Z)
emnist_capitals = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Load and filter EMNIST dataset with 'byclass' split
print("Loading and filtering EMNIST dataset (this may take a moment)...")
emnist_dataset = datasets.EMNIST('../data', split='byclass', train=True, download=True, transform=transform)
filtered_data = [(img, label) for img, label in emnist_dataset if 10 <= label <= 35]  # Filter for capitals only (10-35)
letters_dataset = LettersOnlyDataset(filtered_data)

# Create folder structure and save images
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove existing folder if it exists
os.makedirs(output_folder)

# Create sub-folders for each class
for char in emnist_capitals:
    os.makedirs(os.path.join(output_folder, char), exist_ok=True)

# Save images to corresponding class folders
print("Saving images to folder structure...")
for idx, (image, label) in enumerate(letters_dataset):
    # Reverse normalization
    image = image * 0.3081 + 0.1307
    image = image.squeeze().numpy()

    # Convert to PIL Image
    image = (image * 255).astype(np.uint8)
    pil_image = Image.fromarray(image)

    # Get character from label (adjust by subtracting 10 to map 10-35 to 0-25)
    char = emnist_capitals[label - 10]

    # Save image
    image_path = os.path.join(output_folder, char, f'image_{idx}.png')
    pil_image.save(image_path)

print(f"Dataset saved to {output_folder}")

Loading and filtering EMNIST dataset (this may take a moment)...


100%|██████████| 562M/562M [00:32<00:00, 17.3MB/s]


Saving images to folder structure...
Dataset saved to /content/emnist_capitals_dataset


In [4]:
import os

data_path = '/content/emnist_capitals_dataset'
for char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    folder_path = os.path.join(data_path, char)
    num_images = len(os.listdir(folder_path))
    print(f"Class {char}: {num_images} images")

Class A: 6407 images
Class B: 3878 images
Class C: 10094 images
Class D: 4562 images
Class E: 4934 images
Class F: 9182 images
Class G: 2517 images
Class H: 3152 images
Class I: 11946 images
Class J: 3762 images
Class K: 2468 images
Class L: 5076 images
Class M: 9002 images
Class N: 8237 images
Class O: 24983 images
Class P: 8347 images
Class Q: 2605 images
Class R: 5073 images
Class S: 20764 images
Class T: 9820 images
Class U: 12602 images
Class V: 4637 images
Class W: 4695 images
Class X: 2771 images
Class Y: 4743 images
Class Z: 2701 images


# Download Custom dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Add this at the beginning of main()
import os
import shutil

# Copy dataset from Google Drive to local storage
local_data_path = '/content/letters'
if not os.path.exists(local_data_path):
    shutil.copytree('/content/drive/MyDrive/data/letters', local_data_path)

# Train the model

## Without custom dataset

In [ ]:
import os
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1)  # Input channels: 1 (grayscale), Output channels: 64
        self.conv2 = nn.Conv2d(64, 128, 3, 1)  # Output channels: 128
        self.conv3 = nn.Conv2d(128, 256, 3, 1)  # Third convolutional layer
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.4)
        self.fc1 = nn.Linear(256 * 5 * 5, 512)  # Adjusted for 28x28 input after pooling
        self.fc2 = nn.Linear(512, 256)  # Additional fully connected layer
        self.fc3 = nn.Linear(256, 26)  # Output layer: 26 classes (A-Z)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

# Custom transformation to add Salt-and-Pepper noise
class AddSaltPepperNoise:
    def __init__(self, salt_prob=0.01, pepper_prob=0.01):
        self.salt_prob = salt_prob
        self.pepper_prob = pepper_prob

    def __call__(self, tensor):
        img = tensor.numpy()
        noise = np.random.random(img.shape)
        img[noise < self.salt_prob] = 1.0
        img[noise > (1 - self.pepper_prob)] = 0.0
        return torch.from_numpy(img)

    def __repr__(self):
        return self.__class__.__name__ + '(salt_prob={0}, pepper_prob={1})'.format(self.salt_prob, self.pepper_prob)

# Training Function with Class Weighting
def train(args, model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        if batch_idx % args.log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    train_loss /= len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss, accuracy

# Testing Function with Class Weighting
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    class_correct = [0] * 26
    class_total = [0] * 26

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            batch_loss = criterion(output, target).item()
            test_loss += batch_loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx == 0:
                print(f"Sample outputs (log probs): {output[:5]}")
                print(f"Predicted classes: {pred[:5].view(-1)}")
                print(f"Target classes: {target[:5]}")

            for t, p in zip(target, pred):
                class_correct[t.item()] += (t == p).item()
                class_total[t.item()] += 1

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {avg_test_loss:.6f}, Total sum loss: {test_loss:.6f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')
    for i, char in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
        if class_total[i] > 0:
            print(f'Class {char}: {class_correct[i]}/{class_total[i]} ({100. * class_correct[i] / class_total[i]:.0f}%)')

    return avg_test_loss, accuracy

# Train and Evaluate
def train_and_evaluate(args, model, device, train_loader, test_loader, optimizer, scheduler, criterion):
    test_losses = []
    test_accuracies = []
    train_losses = []
    train_accuracies = []

    best_test_loss = float('inf')
    patience = 8
    no_improvement_count = 0

    # Updated model directory path to Google Drive
    model_dir = f"/content/drive/MyDrive/Capitals_Model_seed_{args.seed}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    for epoch in range(1, args.epochs + 1):
        train_loss, train_accuracy = train(args, model, device, train_loader, optimizer, epoch, criterion)
        test_loss, test_accuracy = test(model, device, test_loader, criterion)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            no_improvement_count = 0
            if args.save_model and epoch > 1:
                model_filename = f"capitals_cnn_epoch:{epoch}_test-accuracy:{test_accuracy:.4f}_test-loss:{test_loss:.4f}.pt"
                model_path = os.path.join(model_dir, model_filename)
                try:
                    torch.save(model.state_dict(), model_path)
                    print(f"Model saved with new best test loss: {best_test_loss:.4f} \n")
                except Exception as e:
                    print(f"Error saving model: {e}")
        else:
            no_improvement_count += 1

        if no_improvement_count >= patience:
            print(f"\nEarly stopping triggered after {epoch} epochs. No improvement in test loss for {patience} consecutive epochs.")
            break

        scheduler.step()

    return train_losses, train_accuracies, test_losses, test_accuracies

# Plot results
def plot_results(train_losses, train_accuracies, test_losses, test_accuracies):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'o-', label='Train Loss')
    plt.plot(epochs, test_losses, 'o-', label='Test Loss')
    plt.title('Loss vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'o-', label='Train Accuracy')
    plt.plot(epochs, test_accuracies, 'o-', label='Test Accuracy')
    plt.title('Accuracy vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Main function
def main():
    from google.colab import drive
    drive.mount('/content/drive')

    random_seed = random.randint(0, 100000)
    print(f"Using random seed: {random_seed}")

    torch.manual_seed(random_seed)
    random.seed(random_seed)

    args = argparse.Namespace(
        batch_size=64,
        test_batch_size=128,
        epochs=100,
        lr=1,
        gamma=0.9,
        no_cuda=False,
        no_mps=False,
        dry_run=False,
        seed=random_seed,
        log_interval=10,
        save_model=True
    )

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}

    if use_cuda:
        cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform_train = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.RandomAffine(degrees=(-5, 5), translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1)], p=0.3),
        transforms.RandomApply([AddSaltPepperNoise(salt_prob=0.01, pepper_prob=0.01)], p=0.2)
    ])

    transform_test = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    data_path = '/content/emnist_capitals_dataset'
    capitals_dataset = datasets.ImageFolder(root=data_path, transform=transform_train)

    train_size = int(0.8 * len(capitals_dataset))
    test_size = len(capitals_dataset) - train_size
    train_dataset, test_dataset = random_split(capitals_dataset, [train_size, test_size])

    test_dataset.dataset.transform = transform_test

    class_counts = [6407, 3878, 10094, 4562, 4934, 9182, 2517, 3152, 11946, 3762, 2468, 5076,
                    9002, 8237, 24983, 8347, 2605, 5073, 20764, 9820, 12602, 4637, 4695, 2771, 4743, 2701]
    class_weights = torch.tensor([1.0 / count for count in class_counts], dtype=torch.float).to(device)
    criterion = nn.NLLLoss(weight=class_weights, reduction='sum')

    train_loader = DataLoader(train_dataset, **train_kwargs)
    test_loader = DataLoader(test_dataset, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

    train_losses, train_accuracies, test_losses, test_accuracies = train_and_evaluate(
        args, model, device, train_loader, test_loader, optimizer, scheduler, criterion
    )

    plot_results(train_losses, train_accuracies, test_losses, test_accuracies)

if __name__ == '__main__':
    main()

串流輸出內容已截斷至最後 5000 行。
Train Epoch: 4 [87040/151166 (58%)]	Loss: 0.001285
Train Epoch: 4 [87680/151166 (58%)]	Loss: 0.001464
Train Epoch: 4 [88320/151166 (58%)]	Loss: 0.001075
Train Epoch: 4 [88960/151166 (59%)]	Loss: 0.003061
Train Epoch: 4 [89600/151166 (59%)]	Loss: 0.000767
Train Epoch: 4 [90240/151166 (60%)]	Loss: 0.001824
Train Epoch: 4 [90880/151166 (60%)]	Loss: 0.000905
Train Epoch: 4 [91520/151166 (61%)]	Loss: 0.001543
Train Epoch: 4 [92160/151166 (61%)]	Loss: 0.000584
Train Epoch: 4 [92800/151166 (61%)]	Loss: 0.000982
Train Epoch: 4 [93440/151166 (62%)]	Loss: 0.000650
Train Epoch: 4 [94080/151166 (62%)]	Loss: 0.001735
Train Epoch: 4 [94720/151166 (63%)]	Loss: 0.003665
Train Epoch: 4 [95360/151166 (63%)]	Loss: 0.001738
Train Epoch: 4 [96000/151166 (64%)]	Loss: 0.002788
Train Epoch: 4 [96640/151166 (64%)]	Loss: 0.000420
Train Epoch: 4 [97280/151166 (64%)]	Loss: 0.000735
Train Epoch: 4 [97920/151166 (65%)]	Loss: 0.000902
Train Epoch: 4 [98560/151166 (65%)]	Loss: 0.000962
Train Epoc

KeyboardInterrupt: 

## Add the custom dataset

In [11]:
import os
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

# Define the neural network (unchanged)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1)
        self.conv2 = nn.Conv2d(64, 128, 3, 1)
        self.conv3 = nn.Conv2d(128, 256, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.4)
        self.fc1 = nn.Linear(256 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 26)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

# Custom transformation to add Salt-and-Pepper noise (unchanged)
class AddSaltPepperNoise:
    def __init__(self, salt_prob=0.01, pepper_prob=0.01):
        self.salt_prob = salt_prob
        self.pepper_prob = pepper_prob

    def __call__(self, tensor):
        img = tensor.numpy()
        noise = np.random.random(img.shape)
        img[noise < self.salt_prob] = 1.0
        img[noise > (1 - self.pepper_prob)] = 0.0
        return torch.from_numpy(img)

    def __repr__(self):
        return self.__class__.__name__ + '(salt_prob={0}, pepper_prob={1})'.format(self.salt_prob, self.pepper_prob)

# Training Function (unchanged)
def train(args, model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        if batch_idx % args.log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    train_loss /= len(train_loader)
    accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss, accuracy

# Testing Function (unchanged)
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    class_correct = [0] * 26
    class_total = [0] * 26

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            batch_loss = criterion(output, target).item()
            test_loss += batch_loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx == 0:
                print(f"Sample outputs (log probs): {output[:5]}")
                print(f"Predicted classes: {pred[:5].view(-1)}")
                print(f"Target classes: {target[:5]}")

            for t, p in zip(target, pred):
                class_correct[t.item()] += (t == p).item()
                class_total[t.item()] += 1

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {avg_test_loss:.6f}, Total sum loss: {test_loss:.6f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')
    for i, char in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
        if class_total[i] > 0:
            print(f'Class {char}: {class_correct[i]}/{class_total[i]} ({100. * class_correct[i] / class_total[i]:.0f}%)')

    return avg_test_loss, accuracy

# Train and Evaluate (unchanged)
def train_and_evaluate(args, model, device, train_loader, test_loader, optimizer, scheduler, criterion):
    test_losses = []
    test_accuracies = []
    train_losses = []
    train_accuracies = []

    best_test_loss = float('inf')
    patience = 8
    no_improvement_count = 0

    model_dir = f"/content/drive/MyDrive/Capitals_Model_seed_{args.seed}"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    for epoch in range(1, args.epochs + 1):
        train_loss, train_accuracy = train(args, model, device, train_loader, optimizer, epoch, criterion)
        test_loss, test_accuracy = test(model, device, test_loader, criterion)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            no_improvement_count = 0
            if args.save_model and epoch > 1:
                model_filename = f"capitals_cnn_epoch:{epoch}_test-accuracy:{test_accuracy:.4f}_test-loss:{test_loss:.4f}.pt"
                model_path = os.path.join(model_dir, model_filename)
                try:
                    torch.save(model.state_dict(), model_path)
                    print(f"Model saved with new best test loss: {best_test_loss:.4f} \n")
                except Exception as e:
                    print(f"Error saving model: {e}")
        else:
            no_improvement_count += 1

        if no_improvement_count >= patience:
            print(f"\nEarly stopping triggered after {epoch} epochs. No improvement in test loss for {patience} consecutive epochs.")
            break

        scheduler.step()

    return train_losses, train_accuracies, test_losses, test_accuracies

# Plot results (unchanged)
def plot_results(train_losses, train_accuracies, test_losses, test_accuracies):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'o-', label='Train Loss')
    plt.plot(epochs, test_losses, 'o-', label='Test Loss')
    plt.title('Loss vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'o-', label='Train Accuracy')
    plt.plot(epochs, test_accuracies, 'o-', label='Test Accuracy')
    plt.title('Accuracy vs. Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Main function (modified)
def main():
    from google.colab import drive
    drive.mount('/content/drive')

    random_seed = random.randint(0, 100000)
    print(f"Using random seed: {random_seed}")

    torch.manual_seed(random_seed)
    random.seed(random_seed)

    args = argparse.Namespace(
        batch_size=64,
        test_batch_size=128,
        epochs=100,
        lr=1,
        gamma=0.9,
        no_cuda=False,
        no_mps=False,
        dry_run=False,
        seed=random_seed,
        log_interval=10,
        save_model=True
    )

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}

    if use_cuda:
        cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform_train = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.RandomAffine(degrees=(-5, 5), translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1)], p=0.3),
        transforms.RandomApply([AddSaltPepperNoise(salt_prob=0.01, pepper_prob=0.01)], p=0.2)
    ])

    transform_test = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load both datasets
    emnist_data_path = '/content/emnist_capitals_dataset'
    letters_data_path = '/content/letters'

    emnist_dataset = datasets.ImageFolder(root=emnist_data_path, transform=transform_train)
    letters_dataset = datasets.ImageFolder(root=letters_data_path, transform=transform_train)

    # Combine the datasets
    combined_dataset = ConcatDataset([emnist_dataset, letters_dataset])

    # Split into train and test
    train_size = int(0.8 * len(combined_dataset))
    test_size = len(combined_dataset) - train_size
    train_dataset, test_dataset = random_split(combined_dataset, [train_size, test_size])

    # Apply test transform to test dataset
    test_dataset.dataset.datasets[0].transform = transform_test  # EMNIST
    test_dataset.dataset.datasets[1].transform = transform_test  # Letters

    # Class counts: EMNIST + Letters (200 images per class for Letters)
    emnist_class_counts = [6407, 3878, 10094, 4562, 4934, 9182, 2517, 3152, 11946, 3762, 2468, 5076,
                           9002, 8237, 24983, 8347, 2605, 5073, 20764, 9820, 12602, 4637, 4695, 2771, 4743, 2701]
    letters_class_counts = [200] * 26  # 200 images per class
    combined_class_counts = [emnist + letters for emnist, letters in zip(emnist_class_counts, letters_class_counts)]

    # Calculate class weights
    class_weights = torch.tensor([1.0 / count for count in combined_class_counts], dtype=torch.float).to(device)
    criterion = nn.NLLLoss(weight=class_weights, reduction='sum')

    # Data loaders
    train_loader = DataLoader(train_dataset, **train_kwargs)
    test_loader = DataLoader(test_dataset, **test_kwargs)

    # Model, optimizer, and scheduler
    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

    # Train and evaluate
    train_losses, train_accuracies, test_losses, test_accuracies = train_and_evaluate(
        args, model, device, train_loader, test_loader, optimizer, scheduler, criterion
    )

    # Plot results
    plot_results(train_losses, train_accuracies, test_losses, test_accuracies)

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using random seed: 13692
Train Epoch: 1 [0/155326 (0%)]	Loss: 0.028068
Train Epoch: 1 [640/155326 (0%)]	Loss: 0.029251
Train Epoch: 1 [1280/155326 (1%)]	Loss: 0.024318
Train Epoch: 1 [1920/155326 (1%)]	Loss: 0.027063
Train Epoch: 1 [2560/155326 (2%)]	Loss: 0.027753
Train Epoch: 1 [3200/155326 (2%)]	Loss: 0.030128
Train Epoch: 1 [3840/155326 (2%)]	Loss: 0.028360
Train Epoch: 1 [4480/155326 (3%)]	Loss: 0.030302
Train Epoch: 1 [5120/155326 (3%)]	Loss: 0.029721
Train Epoch: 1 [5760/155326 (4%)]	Loss: 0.026208
Train Epoch: 1 [6400/155326 (4%)]	Loss: 0.026670
Train Epoch: 1 [7040/155326 (5%)]	Loss: 0.026726
Train Epoch: 1 [7680/155326 (5%)]	Loss: 0.025078
Train Epoch: 1 [8320/155326 (5%)]	Loss: 0.023940
Train Epoch: 1 [8960/155326 (6%)]	Loss: 0.030562
Train Epoch: 1 [9600/155326 (6%)]	Loss: 0.026938
Train Epoch: 1 [10240/155326 (7%)]	Loss: 0.030870
Train Epoch: 1 [

KeyboardInterrupt: 